In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from tqdm import tqdm
import pandas  as pd
import xgboost as xgb

In [2]:
train_data = pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')
test_data  = pd.read_csv('../input/tabular-playground-series-jan-2021/test.csv')
train_data.head()

,id,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
0,1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
1,3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
2,4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
3,5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
4,8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [3]:
features = ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14']

X_train = train_data[features]
y_train = train_data["target"]
final_X_test = test_data[features]

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1)

In [5]:
regressor = xgb.XGBRegressor(n_estimators =4000,colsample_bytree=0.5,alpha=0.01563,learning_rate=0.01,max_depth=15,reg_lambda=0.003,subsample=0.7,metric_period=100,silent=1)

regressor.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)])

[06:06:49] WARNING: ../src/learner.cc:516: 
Parameters: { metric_period, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:7.36324
Will train until validation_0-rmse hasn't improved in 6 rounds.
[1]	validation_0-rmse:7.29028
[2]	validation_0-rmse:7.21806
[3]	validation_0-rmse:7.14653
[4]	validation_0-rmse:7.07573
[5]	validation_0-rmse:7.00569
[6]	validation_0-rmse:6.93635
[7]	validation_0-rmse:6.86768
[8]	validation_0-rmse:6.79971
[9]	validation_0-rmse:6.73242
[10]	validation_0-rmse:6.66586
[11]	validation_0-rmse:6.59993
[12]	validation_0-rmse:6.53468
[13]	validation_0-rmse:6.47009
[14]	validation_0-rmse:6.40610
[15]	validation_0-rmse:6.34284
[16]	validation_0-rmse:6.28021
[17]	validation_0-rmse:6.21819
[18]	validation_0-rmse:6.15682
[19]	validation_

XGBRegressor(alpha=0.01563, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=15, metric_period=100, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=4000, n_jobs=0,
             num_parallel_tree=1, random_state=0, reg_alpha=0.0156299993,
             reg_lambda=0.003, scale_pos_weight=1, silent=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [6]:
y_xgb_pred = regressor.predict(final_X_test)

In [7]:
output1 = pd.DataFrame({"id":test_data.id, "target":y_xgb_pred})
output1.to_csv('xgb.csv', index=False)

In [8]:
params={'random_state': 33,'n_estimators':5000,
 'min_data_per_group': 5,
 'boosting_type': 'gbdt',
 'num_leaves': 256,
 'max_dept': -1,
 'learning_rate': 0.02,
 'subsample_for_bin': 200000,
 'lambda_l1': 1.074622455507616e-05,
 'lambda_l2': 2.0521330798729704e-06,
 'n_jobs': -1,
 'cat_smooth': 1.0,
 'silent': True,
 'importance_type': 'split',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'bagging_fraction': 0.8206341150202605,
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.001,
 'bagging_freq': 6,
 'feature_fraction': 0.5,
 'min_gain_to_split': 0.0,
 'min_child_samples': 20}

In [9]:
from lightgbm import LGBMRegressor

lgb_model = LGBMRegressor(**params)
lgb_model.fit(X_train, y_train, eval_set=(X_test, y_test),early_stopping_rounds = 50, verbose = 0)

LGBMRegressor(bagging_fraction=0.8206341150202605, bagging_freq=6,
              cat_smooth=1.0, feature_fraction=0.5, feature_pre_filter=False,
              lambda_l1=1.074622455507616e-05, lambda_l2=2.0521330798729704e-06,
              learning_rate=0.02, max_dept=-1, metric='rmse',
              min_data_in_leaf=100, min_data_per_group=5, min_gain_to_split=0.0,
              min_sum_hessian_in_leaf=0.001, n_estimators=5000, num_leaves=256,
              random_state=33)

In [10]:
y_pred_lgb = lgb_model.predict(final_X_test)

In [11]:
output2 = pd.DataFrame({"id":test_data.id, "target":y_pred_lgb})
output2.to_csv('lgbm.csv', index=False)

In [12]:
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=800,
                             learning_rate=0.05,
                             depth=10,
                             reg_lambda=10,
                             eval_metric='RMSE',
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)

In [13]:
cb_model.fit(X_train, y_train,eval_set=(X_test, y_test),use_best_model=True,verbose=50)

0:	learn: 0.7316980	test: 0.7323344	best: 0.7323344 (0)	total: 179ms	remaining: 2m 22s
50:	learn: 0.7108046	test: 0.7143758	best: 0.7143758 (50)	total: 6.06s	remaining: 1m 28s
100:	learn: 0.7044291	test: 0.7105012	best: 0.7105012 (100)	total: 12.9s	remaining: 1m 29s
150:	learn: 0.7000979	test: 0.7082751	best: 0.7082751 (150)	total: 18.6s	remaining: 1m 20s
200:	learn: 0.6964453	test: 0.7067988	best: 0.7067988 (200)	total: 24.8s	remaining: 1m 14s
250:	learn: 0.6933032	test: 0.7056717	best: 0.7056717 (250)	total: 31.5s	remaining: 1m 8s
300:	learn: 0.6903357	test: 0.7046570	best: 0.7046570 (300)	total: 38s	remaining: 1m 2s
350:	learn: 0.6873177	test: 0.7037824	best: 0.7037824 (350)	total: 44.8s	remaining: 57.3s
400:	learn: 0.6843673	test: 0.7031336	best: 0.7031250 (398)	total: 50.5s	remaining: 50.3s
450:	learn: 0.6816921	test: 0.7026932	best: 0.7026932 (450)	total: 56.7s	remaining: 43.9s
500:	learn: 0.6789260	test: 0.7021898	best: 0.7021776 (498)	total: 1m 2s	remaining: 37.3s
550:	learn: 0

In [14]:
cat_pred =cb_model.predict(final_X_test)

In [15]:
output3 = pd.DataFrame({"id":test_data.id, "target":cat_pred})
output3.to_csv('cat.csv', index=False)

In [16]:
results = (y_pred_lgb + y_xgb_pred + cat_pred) / 3 

output = pd.DataFrame({"id":test_data.id, "target":results})
output.to_csv('submission.csv', index=False)

In [17]:
output

,id,target
0,0,7.975751
1,2,7.890110
2,6,7.903181
3,7,8.269057
4,10,8.160069
...,...,...
199995,499984,8.154576
199996,499985,8.143500
199997,499987,8.183156
199998,499988,8.045129
